In [2]:
from train_yh import *
from utils_yh import *

import argparse
import os
import gc
import time
import json
import random
import warnings
from pprint import pprint

import numpy as np
import pandas as pd
from einops import rearrange, reduce, repeat
import tez
import torch
import torch.nn as nn
from sklearn import metrics
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForMaskedLM

import transformers

import copy
import warnings
import os

import numpy as np
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoModelForMaskedLM
from joblib import Parallel, delayed 
from tez import enums
from tez.callbacks import Callback
from tqdm import tqdm

import re
import nltk

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enkeejunior1 (use `wandb login --relogin` to force relogin)


In [3]:
os.chdir('/home/enkeejunior1/kaggle_feedbackprize/fb-sbert-yh')

In [4]:
# setting
class set_args_submission:
    def __init__(self):
        self.model: str = 'longformer-base-4096'
        self.sbert: bool = False
        self.output: str = '../model'
        self.input: str = '../input'
        self.max_len: int = 4096
        self.valid_batch_size: int = 4
        self.kfold = 5
            
args = set_args_submission()

In [5]:
def back_translation(df, src2tgt_tokenizer, src2tgt, tgt2src_tokenizer, tgt2src, args, num_jobs):
    total_ids = df["id"].unique()
    total_ids_splits = np.array_split(total_ids, num_jobs)

    results = Parallel(n_jobs=num_jobs, backend="multiprocessing")(
        delayed(_back_translation_helper)(idx, src2tgt_tokenizer, src2tgt, tgt2src_tokenizer, tgt2src, args) for idx in total_ids_splits
    )
    
    return

def _back_translation_helper(total_ids, src2tgt_tokenizer, src2tgt, tgt2src_tokenizer, tgt2src, args):
    for idx in tqdm(total_ids):
        filename = os.path.join(args.input, "train", idx + ".txt")
        with open(filename, "r") as f:
            text = f.read()
        
        # translation
        input_ids_split = [src2tgt_tokenizer.encode(sent, return_tensors="pt") for sent in nltk.sent_tokenize(text)]
        outputs_split = []
        for input_ids in input_ids_split:
            outputs = src2tgt.generate(input_ids)
            outputs_split.append(outputs)
        decode_split = []
        for outputs in outputs_split:
            decode_split.append(src2tgt_tokenizer.decode(outputs.squeeze(), skip_special_tokens = True))
        
        # back-translation
        input_ids_split = [tgt2src_tokenizer.encode(sent, return_tensors="pt") for sent in decode_split]
        outputs_split = []
        for input_ids in input_ids_split:
            outputs = tgt2src.generate(input_ids)
            outputs_split.append(outputs)
        decode_split = []
        for outputs in outputs_split:
            decode_split.append(tgt2src_tokenizer.decode(outputs.squeeze(), skip_special_tokens = True))
        
        filename = os.path.join(args.input, "aug_back_translation", idx + ".txt")
        with open(filename, 'w') as f:
            f.write(' '.join(decode_split))
    return 

In [6]:
NUM_JOBS = 12

# read kfold data
if os.path.isfile(args.input + '/' + f'train_{args.kfold}folds.csv'):
    df = pd.read_csv(args.input + '/' + f'train_{args.kfold}folds.csv')
else:
    df = pd.read_csv(args.input_path + '/' + f'train.csv')
    df = get_stratified_fold(df, args)

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
src2tgt_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
src2tgt = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")

tgt2src_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
tgt2src = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

src2tgt.eval()
tgt2src.eval()

In [9]:
import psutil
n_jobs = psutil.cpu_count()

In [ ]:
sys.ex

In [ ]:
back_translation(df, src2tgt_tokenizer, src2tgt, tgt2src_tokenizer, tgt2src, args, num_jobs = 1)

  3%|▎         | 535/15594 [3:36:20<221:16:04, 52.90s/it]